In [ ]:
#leave this untouched
#define gsheet_api_check to obtain permission
import pickle
import os.path
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
def gsheet_api_check(SCOPES):
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secrets.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

In [ ]:
#leave this untouched
#define pull_sheet_date to import spreadsheet from google sheets
from googleapiclient.discovery import build
def pull_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_PULL):
    creds = gsheet_api_check(SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=DATA_TO_PULL).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range=DATA_TO_PULL).execute()
        data = rows.get('values')
        print("COMPLETE: Data copied")
        return data

In [ ]:
#import the desired spreadsheet from google sheets
#input the spreadsheet id for the desired spreadsheet

import pandas as pd
SCOPES = ['https://www.googleapis.com/auth/spreadsheets'] #leave this untouched
SPREADSHEET_ID = '' #get id from hyperlink and insert here; between "https://docs.google.com/spreadsheets/d/" and "/edit"
DATA_TO_PULL = 'Form Responses 1' #name of the tab to read
data = pull_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_PULL)
df = pd.DataFrame(data[1:], columns=data[0]) #put spreadsheet into dataframe

In [ ]:
#get the last row of spreadsheet
df2 = df.tail(1)

In [ ]:
#add dataframe into a tab in desired google sheets
from googleapiclient import discovery
def append_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_PULL):
    creds = gsheet_api_check(SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().append(
        valueInputOption = 'RAW',
        spreadsheetId=SPREADSHEET_ID,
        body = dict(majorDimension = 'ROWS',
        values = df2.T.reset_index().values.tolist()),
        range=DATA_TO_APPEND).execute()

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SPREADSHEET_ID = '' #get id from hyperlink and insert here; the code between "https://docs.google.com/spreadsheets/d/" and "/edit"
DATA_TO_APPEND = 'Speaker form entry!A6'
append_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_APPEND)
